## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-15-13-00-18 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/15/us/sup...
1,2025-10-15-13-00-00 +0000,missionlocal,Crome Panther,https://missionlocal.org/2025/10/sf-photos-cro...
2,2025-10-15-12-57-59 +0000,nyt,"What Are Tomahawk Missiles, and Why Might Trum...",https://www.nytimes.com/2025/10/15/us/politics...
3,2025-10-15-12-51-58 +0000,bbc,When is the Budget and what might be in it?,https://www.bbc.com/news/articles/cj4w44w42j5o...
4,2025-10-15-12-46-00 +0000,wsj,S&P Global Buying Data and Analytics Firm With...,https://www.wsj.com/business/deals/s-p-global-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,48
107,gaza,18
370,hamas,18
20,new,16
414,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
294,2025-10-14-17-30-22 +0000,nypost,"Gaza peace process ‘IS NOT DONE,’ Trump warns ...",https://nypost.com/2025/10/14/us-news/gaza-pea...,116
333,2025-10-14-15-25-15 +0000,nypost,Hamas won’t give up arms or Gaza control — des...,https://nypost.com/2025/10/14/world-news/hamas...,116
348,2025-10-14-14-24-22 +0000,nypost,Kamala Harris refuses to name Trump as she lau...,https://nypost.com/2025/10/14/us-news/kamala-h...,115
268,2025-10-14-18-52-56 +0000,nypost,"Hamas will disarm or ‘we will disarm them,’ Tr...",https://nypost.com/2025/10/14/us-news/hamas-wi...,109
86,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
294,116,2025-10-14-17-30-22 +0000,nypost,"Gaza peace process ‘IS NOT DONE,’ Trump warns ...",https://nypost.com/2025/10/14/us-news/gaza-pea...
63,62,2025-10-15-10-00-00 +0000,nypost,Dr. Keith Ablow tells ‘Pod Force One’ why he f...,https://nypost.com/2025/10/15/us-news/dr-keith...
86,60,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...
254,45,2025-10-14-19-42-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
23,44,2025-10-15-12-11-39 +0000,bbc,Government will publish evidence in collapsed ...,https://www.bbc.com/news/articles/cql9v6x4wpzo...
348,40,2025-10-14-14-24-22 +0000,nypost,Kamala Harris refuses to name Trump as she lau...,https://nypost.com/2025/10/14/us-news/kamala-h...
270,38,2025-10-14-18-38-00 +0000,wsj,Fed Chair Powell left the central bank on trac...,https://www.wsj.com/economy/central-banking/po...
255,36,2025-10-14-19-38-00 +0000,wsj,"Bouygues Telecom, Free-iliad, Orange Offer to ...",https://www.wsj.com/business/telecom/bouygues-...
261,36,2025-10-14-19-08-39 +0000,bbc,Israel identifies bodies of first four dead ho...,https://www.bbc.com/news/articles/cx27ye11ey2o...
192,33,2025-10-14-22-31-05 +0000,nypost,Trump honors late Charlie Kirk with Medal of F...,https://nypost.com/2025/10/14/us-news/trump-ho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
